In [25]:
# Housekeeping - This cell shortens the traceback of error messages
# unless you're on mybinder.org 

import json 
from pprint import pprint
import sys
ipython = get_ipython()
unhide_traceback = None

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

if not unhide_traceback:
    unhide_traceback = ipython.showtraceback

ipython.showtraceback = hide_traceback   # or:  ipython.showtraceback = unhide_traceback



Pythoncamp 2020 Session Martin Borus, Twitter: @mborus

Translated - Originally done in German

# GLOM

by Mahmoud Hashemi

## "If you have nested data, you need Glom!"

https://github.com/mahmoud/glom 





# Main features of Glom

- Path based access
- Declarative data transformation 
- Easy to read helpful error messages
- comes with debugging features


## Preparation for this notebook

pip install glom  # current version 20.5.0

pip install requests

Full docs and tutorial at https://glom.readthedocs.io,
some examples from this tutorial are included here

# 1. Normal Python

These are some examples on how to access nested data with regular Python

In [26]:
data = {'a': {'b': {'c': 'd'}}}

In [27]:
data['a']['b']['c']

'd'

In [28]:
data2 = {'a': {'b': None}}

In [29]:
data2['a']['b']['c']

TypeError: 'NoneType' object is not subscriptable

# 2. Glom 

Here's how you do the same thing in Glom, using the data above

In [30]:
from glom import glom

In [31]:
glom(data, 'a.b.c')

'd'

In [32]:
# this here will product an error message
glom(data2, 'a.b.c')


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\martin\src\glom\glom\core.py", line 1212, in _t_eval
    cur = get(cur, arg)
AttributeError: 'NoneType' object has no attribute 'c'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Python38\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-32-c52c6938e1d9>", line 2, in <module>
    glom(data2, 'a.b.c')
  File "c:\users\martin\src\glom\glom\core.py", line 1827, in glom
    ret = _glom(target, spec, scope)
  File "c:\users\martin\src\glom\glom\core.py", line 1848, in _glom
    return scope[MODE](target, spec, scope)
  File "c:\users\martin\src\glom\glom\core.py", line 1859, in AUTO
    return Path.from_text(spec).glomit(target, scope)
  File "c:\users\martin\src\glom\glom\core.py", line 373, in glomit
    return _t_eval(target, self.path_t, scope)
  File "c:\users\marti

PathAccessError: could not access 'c', part 2 of Path('a', 'b', 'c'), got error: AttributeError("'NoneType' object has no attribute 'c'")

In [33]:
# Glom is great for catching errors

from glom import GlomError, PathAccessError

try:
    glom(data2, 'a.b.c')
except PathAccessError as e:
    print(e)

could not access 'c', part 2 of Path('a', 'b', 'c'), got error: AttributeError("'NoneType' object has no attribute 'c'")


In [34]:
try:
    glom(data2, 'a.b.c')
except AttributeError as e:
    print(e)

could not access 'c', part 2 of Path('a', 'b', 'c'), got error: AttributeError("'NoneType' object has no attribute 'c'")


In [35]:
try:
    glom(data2, 'a.b.c')
except GlomError as e:
    print(e)

could not access 'c', part 2 of Path('a', 'b', 'c'), got error: AttributeError("'NoneType' object has no attribute 'c'")


## You can use Glom with lists

In [36]:
data = [1, [2 , 3, 4] , 3, 4 ,5]

In [37]:
glom(data, '1.0')

2

## You can use Glom with objects !1!!

In [41]:
class MyClass:
    def __init__(self):
        self.my_hallo = "hello!"
        self.my_world = "world!"
       
myvar = MyClass()        

In [42]:
glom(myvar, 'my_world')

'world!'

# 3. Target & Spec
- the "Target" is the data, (list, dict, object)
- the "Spec" is the wanted result

In [43]:
target = {
     'galaxy': {
        'system': {
            'planet': 'jupiter'
         }
    }
}

spec = 'galaxy.system.planet'

glom(target, spec)

'jupiter'

In [44]:
target = {
    'system': {
        'planets': [
            {'name': 'earth', 'moons': 1},
            {'name': 'jupiter', 'moons': 69}
        ]
    }
}

spec = {
     'names': ('system.planets', ['name']),
     'moons': ('system.planets', ['moons'])
}

glom(target, spec)

{'names': ['earth', 'jupiter'], 'moons': [1, 69]}

In [45]:
target = {
     'system': {
         'planets': [
            {
                'name': 'earth',
                'moons': [
                    {'name': 'luna'}
                ]
            },
            {
                'name': 'jupiter',
                'moons': [
                    {'name': 'io'},
                    {'name': 'europa'}
                ]
            }
        ]
    }
}

In [46]:

spec = {
    'planet_names': ('system.planets', ['name']),
    'moon_names': ('system.planets', [('moons', ['name'],  )])
}
pprint(glom(target, spec))

{'moon_names': [['luna'], ['io', 'europa']],
 'planet_names': ['earth', 'jupiter']}


In [48]:
# Coalesce: Try the first, if it fails, try the next, and so on....

from glom import Coalesce

target = {
     'system': {
         'planets': [
             {'name': 'earth', 'moons': 1},
             {'name': 'jupiter', 'moons': 69}
         ]
     }
}

spec = {
     'planets': (Coalesce('system.planets', 'system.dwarf_planets'), ['name']),
     'moons': (Coalesce('system.planets', 'system.dwarf_planets'), ['moons'])
}

glom(target, spec)

{'planets': ['earth', 'jupiter'], 'moons': [1, 69]}

In [49]:
target = {
     'system': {
         'dwarf_planets': [
             {'name': 'pluto', 'moons': 5},
             {'name': 'ceres', 'moons': 0},
         ]
     }
 }
glom(target, spec)

{'planets': ['pluto', 'ceres'], 'moons': [5, 0]}

In [53]:
# use python functions or lambdas inside the code

target = {
     'system': {
         'planets': [
             {'name': 'earth', 'moons': 1},
             {'name': 'jupiter', 'moons': 69}
         ]
     }
}

print(glom(target, {'moon_count': ('system.planets', ['moons'], sum)}))
    
print(glom(target, {'moon_count': ('system.planets', ['moons'], lambda x: sum(x))}))

{'moon_count': 70}
{'moon_count': 70}


In [56]:
# glom a class, subclass structure

class MySubClass:
    def __init__(self):
        self.my_hey = "Hey!"

class MyClass:
    def __init__(self):
        self.my_hello = "hello!"
        self.my_world = "world!"
        self.my_heylist = [MySubClass()] * 6

        
myvar = MyClass()  

In [59]:
from glom import Iter
spec = {'hello': 'my_hello', 'world': 'my_world', 'heylist': ('my_heylist', ['my_hey'])}

glom(myvar, spec)

{'hello': 'hello!',
 'world': 'world!',
 'heylist': ['Hey!', 'Hey!', 'Hey!', 'Hey!', 'Hey!', 'Hey!']}

# 4. Flatten and merge data

Convert a list of lists & list of dicts to a list or dict

In [65]:
from glom import Flatten, Merge

In [66]:
data = [[1, 2], [3], [4], [], [5]]

In [67]:
glom(data, Flatten())

[1, 2, 3, 4, 5]

In [68]:
data = [{'hello': 'world'}, {'hello2': 'world2'}]

In [69]:
glom(data, Merge())

{'hello': 'world', 'hello2': 'world2'}

# 5. from the tutorial: Convert objects into output.

This shows a more complicated example on how nested class objects are converted.
Read the official tutorial on what happens here.

In [70]:
from glom.tutorial import * 

In [71]:
contact = Contact('Julian',
          emails=[Email(email='jlahey@svtp.info')],
                  location='Canada')

In [72]:
contact.save()

In [73]:
contact.primary_email

Email(id=5, email='jlahey@svtp.info', email_type='personal')

In [74]:
contact.add_date

datetime.datetime(2020, 7, 29, 16, 23, 45, 779009)

In [75]:
contact.id

5

In [76]:
len(Contact.objects.all())

5

In [77]:
Contact.objects.all()

[Contact(id=1, name='Kurt', pref_name='', emails=[Email(id=1, email='kurt@example.com', email_type='personal')], primary_email=Email(id=1, email='kurt@example.com', email_type='personal'), company='', location='Mountain View', add_date=datetime.datetime(2020, 7, 29, 16, 23, 45, 660002)),
 Contact(id=2, name='Sean', pref_name='', emails=[Email(id=2, email='seanboy@example.com', email_type='personal')], primary_email=Email(id=2, email='seanboy@example.com', email_type='personal'), company='D & D Mastering', location='San Jose', add_date=datetime.datetime(2020, 7, 29, 16, 23, 45, 660002)),
 Contact(id=3, name='Matt', pref_name='', emails=[Email(id=3, email='mixtape@homemakelabs.com', email_type='work'), Email(id=4, email='matt@example.com', email_type='personal')], primary_email=Email(id=3, email='mixtape@homemakelabs.com', email_type='work'), company='HomeMake Labs', location='', add_date=datetime.datetime(2020, 7, 29, 16, 23, 45, 660002)),
 Contact(id=4, name='Julian', pref_name='', ema

In [78]:
# without Glom: You can't dump the object to json

json.dumps(Contact.objects.all())

TypeError: Object of type Contact is not JSON serializable

In [80]:
target = Contact.objects.all()

In [81]:
# have a look at the data type of the "add_date" - this is not dumpable
target[0].add_date

datetime.datetime(2020, 7, 29, 16, 23, 45, 660002)

In [85]:
# Note: In this spec, the datetime and integers are converted to string.

spec = {'results': [{'id': 'id',
                      'name': 'name',
                      'add_date': ('add_date', str),
                      'emails': ('emails', [{'id': 'id',
                                            'email': 'email',
                                            'type': 'email_type'}]),
                      'primary_email': Coalesce('primary_email.email', default=None),
                      'pref_name': Coalesce('pref_name', 'name', skip='', default=''),
                      'detail': Coalesce('company',
                                         'location',
                                         ('add_date.year', str),
                                         skip='', default='')}]}

In [88]:
# with Glom: convert your target to somethin you can dump

resp = glom(target, spec)
print(json.dumps(resp, indent=2, sort_keys=True))

{
  "results": [
    {
      "add_date": "2020-07-29 16:23:45.660002",
      "detail": "Mountain View",
      "emails": [
        {
          "email": "kurt@example.com",
          "id": 1,
          "type": "personal"
        }
      ],
      "id": 1,
      "name": "Kurt",
      "pref_name": "Kurt",
      "primary_email": "kurt@example.com"
    },
    {
      "add_date": "2020-07-29 16:23:45.660002",
      "detail": "D & D Mastering",
      "emails": [
        {
          "email": "seanboy@example.com",
          "id": 2,
          "type": "personal"
        }
      ],
      "id": 2,
      "name": "Sean",
      "pref_name": "Sean",
      "primary_email": "seanboy@example.com"
    },
    {
      "add_date": "2020-07-29 16:23:45.660002",
      "detail": "HomeMake Labs",
      "emails": [
        {
          "email": "mixtape@homemakelabs.com",
          "id": 3,
          "type": "work"
        },
        {
          "email": "matt@example.com",
          "id": 4,
          "type": "per

In [91]:
# This is the Flatten command from the previous section, used to get all emails in the result

from glom import Flatten

glom(resp, ('results', ['emails'],
             Flatten(),
             ['email'],
           )
    )


['kurt@example.com',
 'seanboy@example.com',
 'mixtape@homemakelabs.com',
 'matt@example.com',
 'jlahey@svtp.info']

# 6. "T" - the Stunt-Double

T in the spect is a stand in for anything at that position in the spec.
It behaves like the python object it matches. 

In the first example,
it behaves like a list, so T[0] is the first list object

In [92]:
from glom import T

In [93]:
from glom import T, Flatten
# from the data structure above, show the first entry
glom(resp, ('results', T[0]))

{'id': 1,
 'name': 'Kurt',
 'add_date': '2020-07-29 16:23:45.660002',
 'emails': [{'id': 1, 'email': 'kurt@example.com', 'type': 'personal'}],
 'primary_email': 'kurt@example.com',
 'pref_name': 'Kurt',
 'detail': 'Mountain View'}

T also works with Objects. Very nice for Namedtuples.



In [95]:
from collections import namedtuple
HelloWorld = namedtuple('HelloWord','hello,world')
hw = HelloWorld('hello!', 'world!')

In [96]:
glom(hw, T.hello)

'hello!'

In [97]:
glom(hw, (T._asdict(), 'hello'))

'hello!'

# 7. Let's loop with Iter

In [103]:
# let's prepare a list of the Seven Dwarfs

from glom import glom, Iter

target = [
    'Happy',
    'Sneezy',
    'Sleepy',
    'Doc',
    'Bashful',
    'Grumpy',
    'Dopey'
]

In [104]:
# Iter returns a generator type
spec = Iter()
glom(target, spec)

<generator object Iter._iterate at 0x00000000053F52E0>

In [105]:
# Convert the generator into a list with ".all"
spec = Iter().all()
glom(target, spec)

['Happy', 'Sneezy', 'Sleepy', 'Doc', 'Bashful', 'Grumpy', 'Dopey']

In [106]:
# Chunk the group in teams of 2
spec = Iter().chunked(2, fill='Snow White').all()
glom(target, spec)

[['Happy', 'Sneezy'],
 ['Sleepy', 'Doc'],
 ['Bashful', 'Grumpy'],
 ['Dopey', 'Snow White']]

In [108]:
# Find any dwarf that's not doc with a lambda
spec = Iter().filter(lambda x: x != 'Doc').all()
glom(target, spec)

['Happy', 'Sneezy', 'Sleepy', 'Bashful', 'Grumpy', 'Dopey']

In [109]:
# Hey, 3 dwarfs maximum!
spec = Iter().limit(3).all()
glom(target, spec)

['Happy', 'Sneezy', 'Sleepy']

In [110]:
# Find dwarfs 2 - 4 in this 0-based list
spec = Iter().slice(1, 4).all()
glom(target, spec)

['Sneezy', 'Sleepy', 'Doc']

In [111]:
# Only show dwarfs with unique first letters in the names
spec = Iter().unique(T[0]).all()
glom(target, spec)

['Happy', 'Sneezy', 'Doc', 'Bashful', 'Grumpy']

In [113]:
# Take dwarfs until Doc arrives
spec = Iter().takewhile(lambda x: x != 'Doc').all()
glom(target, spec)

['Happy', 'Sneezy', 'Sleepy']

In [114]:
# Show the first dwarf following Doc
spec = Iter().dropwhile(lambda x: x != 'Chef').slice(1, 2).first()
glom(target, spec)

In [116]:
# Split the group around Doc
spec = Iter().split('Doc').all() 
glom(target, spec)

[['Happy', 'Sneezy', 'Sleepy'], ['Bashful', 'Grumpy', 'Dopey']]

In [117]:
# Use a function on all
spec = Iter().map(lambda x:x.lower()).all()
glom(target, spec)


['happy', 'sneezy', 'sleepy', 'doc', 'bashful', 'grumpy', 'dopey']

In [118]:
# or instead of lambda, the stunt double T
spec = Iter().map(T.upper()).all()
glom(target, spec)

['HAPPY', 'SNEEZY', 'SLEEPY', 'DOC', 'BASHFUL', 'GRUMPY', 'DOPEY']

# 8. Literal - assign a fixed value

In [119]:
from glom import Literal
spec = Iter({'Name': T, 'Size': Literal('Dwarflike')}).all()

In [120]:
glom(target, spec)

[{'Name': 'Happy', 'Size': 'Dwarflike'},
 {'Name': 'Sneezy', 'Size': 'Dwarflike'},
 {'Name': 'Sleepy', 'Size': 'Dwarflike'},
 {'Name': 'Doc', 'Size': 'Dwarflike'},
 {'Name': 'Bashful', 'Size': 'Dwarflike'},
 {'Name': 'Grumpy', 'Size': 'Dwarflike'},
 {'Name': 'Dopey', 'Size': 'Dwarflike'}]

# 9. Data Driven

What to do, if a dictionary key has the data

In [121]:
from glom import glom, T, Merge, Iter, Coalesce

target = {
    "pluto": {"moons": 6, "population": None},
    "venus": {"population": {"aliens": 5}},
    "earth": {"moons": 1, "population": {"humans": 7_700_000_000, "aliens": 1}},
}

spec = {
    "moons": (
           T.items(),
           Iter({T[0]: (T[1], Coalesce("moons", default=0))}),
           Merge()
    )
}
        
glom(target, spec)        

{'moons': {'pluto': 6, 'venus': 0, 'earth': 1}}

# 10. Add and remove values


In [122]:
data = {'moons': {'pluto': 6, 'venus': 0, 'earth': 1}}

In [123]:
from glom import Assign, Delete
spec = Assign('moons.saturn', 7)

In [124]:
glom(data, spec)

{'moons': {'pluto': 6, 'venus': 0, 'earth': 1, 'saturn': 7}}

In [125]:
spec = Delete('moons.earth')

In [126]:
glom(data, spec)

{'moons': {'pluto': 6, 'venus': 0, 'saturn': 7}}

In [127]:
spec = Delete('moons.mars', ignore_missing=True)

glom(data, spec)


{'moons': {'pluto': 6, 'venus': 0, 'saturn': 7}}

# 11. Scope / Let

Scope allows you to collect data which would otherwise not be accessable from within a spec.

This is nice if you want to move data inside each element to get a flat structure you can import into a pandas Dataframe

In [131]:
from glom import S, glom, Assign, Spec

target = {'date': '2020-04-01',
 'location': 'A',
 'items': [
     {'name': 'A', 'id': 'A1'},
     {'name': 'B', 'id': 'B1'},
     {'name': 'C', 'id': 'C1'}
]}

spec = ('items', 
        [Assign('date', Spec(S['date']))], 
        [Assign('location', Spec(S['location']))]
       )

glom(target, spec, scope=target)

[{'name': 'A', 'id': 'A1', 'date': '2020-04-01', 'location': 'A'},
 {'name': 'B', 'id': 'B1', 'date': '2020-04-01', 'location': 'A'},
 {'name': 'C', 'id': 'C1', 'date': '2020-04-01', 'location': 'A'}]

The same example with let:
Let writes to the scope at runtime.


In [132]:
from glom import Fill
from glom.core import Let

spec = (
    # Write outer value to scope
    Let(base={"date": "date", "location": "location"}),
    # select just the items
    "items",
    [
        # for every element: add base to element
        (Fill([T, S["base"]]), Merge())
    ]
)

glom(target, spec, scope=target)


[{'name': 'A', 'id': 'A1', 'date': '2020-04-01', 'location': 'A'},
 {'name': 'B', 'id': 'B1', 'date': '2020-04-01', 'location': 'A'},
 {'name': 'C', 'id': 'C1', 'date': '2020-04-01', 'location': 'A'}]

# 12. XML

Simple example on how to glom XML

In [138]:
from glom import Ref

In [139]:
etree2dicts = Ref('ElementTree',
    {"tag": "tag", 
     "text": "text", 
     "attrib": "attrib", 
     "children": (iter, [Ref('ElementTree')])})

In [140]:
html_text = """<html>
  <head>
    <title>the title</title>
  </head>
  <body id="the-body">
    <p>A paragraph</p>
  </body>
</html>"""

In [141]:
from xml.etree import ElementTree
etree = ElementTree.fromstring(html_text)

In [142]:
glom(etree, etree2dicts)

{'tag': 'html',
 'text': '\n  ',
 'attrib': {},
 'children': [{'tag': 'head',
   'text': '\n    ',
   'attrib': {},
   'children': [{'tag': 'title',
     'text': 'the title',
     'attrib': {},
     'children': []}]},
  {'tag': 'body',
   'text': '\n    ',
   'attrib': {'id': 'the-body'},
   'children': [{'tag': 'p',
     'text': 'A paragraph',
     'attrib': {},
     'children': []}]}]}

# 13. Out of time...

other things worth taking a look at.

- Path (A special Path notation for edge cases, where strings don't work)
- Invoke / Call (Use a function)
- Check -> CheckError (Check for Data consistency!)
- Inspect (Debugging helper: Print out whats visible inside the spec)  




# 14. Glom, Comments and the code formatter "Black"

If you use Black on your code (https://pypi.org/project/black/), black may
make your glom spec a lot harder to read.

If your spec is somewhat complicated, add comments to it.

The above used example still works with comments inside

    spec = (
        # Write outer value to scope
        Let(base={"date": "date", "location": "location"}),

        # select just the items
        "items",
        [
            # for every element: add base to element
            (Fill([T, S["base"]]), Merge())
        ]
    )

Without comments, Black will try to save space.

    spec = (
        Let(base={"date": "date", "location": "location"}),
        "items",
        [(Fill([T, S["base"]]), Merge())],
    )



# Practice exercises

### Exercise 1: 

Take the pythoncamp room list at http://borus.de/pythoncamp/event.json Create a dictionary that has all room names in capital letters as a key and the url as a value,.

Wanted result

    {
     'BERLIN': 'https://bbb01.pythoncamp.online/b/rei-hyz-cgm',
     'FLIEGENDER ZIRKUS': 'https://bbb01.pythoncamp.online/b/rei-hyz-cgm',
     'TOKIO': 'https://bbb01.pythoncamp.online/b/rei-hyz-cgm'
     ...
     }

In [143]:
import requests

In [144]:
r = requests.get(r'http://borus.de/pythoncamp/event.json')

In [145]:
r.json()

{'event': 'pythoncamp 2020',
 'update': '2020-04-17 23:23:00',
 'rooms': [{'name': 'Berlin',
   'url': 'https://bbb01.pythoncamp.online/b/rei-hyz-cgm',
   'access_code': '12345'},
  {'name': 'Fliegender Zirkus',
   'url': 'https://bbb01.pythoncamp.online/b/rei-hyz-cgm',
   'access_code': '12345'},
  {'name': 'London',
   'url': 'https://bbb01.pythoncamp.online/b/rei-hyz-cgm',
   'access_code': '12345'},
  {'name': 'New York',
   'url': 'https://bbb01.pythoncamp.online/b/rei-hyz-cgm',
   'access_code': '12345'},
  {'name': 'Paris',
   'url': 'https://bbb01.pythoncamp.online/b/rei-hyz-cgm',
   'access_code': '12345'},
  {'name': 'Plenum',
   'url': 'https://bbb01.pythoncamp.online/b/rei-hyz-cgm',
   'access_code': '12345'},
  {'name': 'Ritter der Kokosnuss',
   'url': 'https://bbb01.pythoncamp.online/b/rei-hyz-cgm',
   'access_code': '12345'},
  {'name': 'Rom',
   'url': 'https://bbb01.pythoncamp.online/b/rei-hyz-cgm',
   'access_code': '12345'},
  {'name': 'Spamalot',
   'url': 'https:/

In [146]:
# write spec here
spec = "... ?"

In [147]:
glom(r.json(), spec)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\martin\src\glom\glom\core.py", line 1212, in _t_eval
    cur = get(cur, arg)
KeyError: ''

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Python38\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-147-7583fad57d0a>", line 1, in <module>
    glom(r.json(), spec)
  File "c:\users\martin\src\glom\glom\core.py", line 1827, in glom
    ret = _glom(target, spec, scope)
  File "c:\users\martin\src\glom\glom\core.py", line 1848, in _glom
    return scope[MODE](target, spec, scope)
  File "c:\users\martin\src\glom\glom\core.py", line 1859, in AUTO
    return Path.from_text(spec).glomit(target, scope)
  File "c:\users\martin\src\glom\glom\core.py", line 373, in glomit
    return _t_eval(target, self.path_t, scope)
  File "c:\users\martin\src\glom\glom\core.py", line 1214, in _

PathAccessError: could not access '', part 0 of Path('', '', '', ' ?'), got error: KeyError('')

### Exercise 2:

Extract data from an online booking result (in German language, sorry)
http://borus.de/pythoncamp/booking_example.json
        
Wanted result:

    {'departure_harbor_outward_trip': 'DKHNB',
     'departure_date_outward_trip': '2019-09-01',
     'departure_time_outward_trip': '14:30',
     'departure_harbor_return_trip': 'DELIS',
     'departure_date_return_trip': '2019-10-02',
     'departure_time_return_trip': '19:25',
     'tickets': ['CAR', 'AD', 'CH']
    }


In [148]:
r = requests.get(r'http://borus.de/pythoncamp/booking_example.json')

In [149]:
r.json()

{'request_ip': '127.0.0.1',
 'request_system': 'homepage',
 'request_session': '<SESSION UUID4>',
 'request_language': 'en',
 'company': 'REDACTED',
 'agency': None,
 'client_no': '123456789',
 'trip_special': None,
 'booking_type': 'VEH',
 'booking_currency': 'EUR',
 'booking_no': None,
 'trips': [{'trip_part': 'OUT',
   'departure_harbor': 'DKHNB',
   'destination_harbor': 'DELIS',
   'date': '2019-09-01',
   'time': '14:30',
   'date_open': False,
   'selected_voyage': 'H-L070145',
   'selected_reservation_area': None,
   'trip_discount percent': None},
  {'trip_part': 'RET',
   'departure_harbor': 'DELIS',
   'destination_harbor': 'DKHNB',
   'date': '2019-10-02',
   'time': '19:25',
   'date_open': False,
   'selected_voyage': 'H-L070170',
   'selected_reservation_area': None,
   'trip_discount percent': None}],
 'tickets': [{'type': 'CAR',
   'token': None,
   'is_vehicle': True,
   'license_plate': 'NF-TEST-01',
   'count': 1,
   'length': 5.0,
   'reservation_area': None,
   'd

In [ ]:
# write the spec here
spec = "... ?"


In [ ]:
glom(r.json(), spec)